In [1]:
import pandas as pd
import numpy as np

In [2]:
#read in the result of query for last 14 days
df_14 = pd.read_csv('/home/jupyter-vladperesad/files/query-14-.csv')

In [5]:
#create a subset that only contains student_ids
df_st = df_14['student_id']

In [7]:
#calculate percent difference in students performance beween most recent class and the class before that
df_pct = df_14 \
    .drop(columns=['student_name',
                    'group_name',
                    'date']) \
    .groupby('student_id') \
    .pct_change(fill_method=None)*100

In [8]:
#drop rows that have NaN value across all columns (these are the students who came once hence the percentage difference cannot be calculated)
df_pct.dropna(axis=0, how='all', inplace=True)

In [9]:
#add to column that contains stidents ids for later merging
df_pct = df_pct.join(df_st, how='inner')

In [12]:
#read in the result of query for last 7 days
df_7 = pd.read_csv('/home/jupyter-vladperesad/files/query-7-.csv')

In [14]:
final_table = df_7.merge(df_pct, how='left', on='student_id', suffixes=('_act','_pct'))

In [40]:
for index, row in final_table.iterrows():
    date = row['date']
    etc = row['student_id']
    msg = f''' Report from {date}
    Student id: {etc}'''
    print(msg)  

 Report from 2024-02-29
    Student id: 4
 Report from 2024-02-29
    Student id: 5
 Report from 2024-02-29
    Student id: 6
 Report from 2024-02-29
    Student id: 7
 Report from 2024-02-29
    Student id: 8
 Report from 2024-02-29
    Student id: 9
 Report from 2024-03-01
    Student id: 11
 Report from 2024-03-01
    Student id: 12
 Report from 2024-03-01
    Student id: 13
 Report from 2024-03-01
    Student id: 14
 Report from 2024-03-02
    Student id: 16
 Report from 2024-03-02
    Student id: 17
 Report from 2024-03-02
    Student id: 19
 Report from 2024-03-02
    Student id: 20
 Report from 2024-03-02
    Student id: 21
 Report from 2024-03-02
    Student id: 22
 Report from 2024-03-02
    Student id: 23
 Report from 2024-03-02
    Student id: 24
 Report from 2024-03-02
    Student id: 25
 Report from 2024-03-02
    Student id: 26
 Report from 2024-03-02
    Student id: 27
 Report from 2024-03-02
    Student id: 28
 Report from 2024-03-02
    Student id: 29
 Report from 2024